In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch

In [4]:
df = pd.read_csv("../data/phase_train.csv")
df = df.sort_values(["game_episode", "time_seconds"]).reset_index(drop=True)

episodes = []
targets = []
count = 0

print(len(df.groupby(["game_episode", "team_id"])))

for _, g in tqdm(df.groupby(["game_episode", "team_id"])):
    g = g.reset_index(drop=True)
    if len(g) < 2:
        continue

    # 정규화된 좌표 준비
#     sx = g["start_x"].values / 105.0
#     sy = g["start_y"].values / 68.0
#     ex = g["end_x"].values   / 105.0
#     ey = g["end_y"].values   / 68.0

#     coords = []
#     for i in range(len(g)):
#         # 항상 start는 들어감
#         coords.append([sx[i], sy[i]])
#         # 마지막 행 이전까지만 end를 넣음 (마지막 end는 타깃이므로)
#         if i < len(g) - 1:
#             coords.append([ex[i], ey[i]])

#     seq = np.array(coords, dtype="float32")        # [T, 2]
#     target = np.array([ex[-1], ey[-1]], dtype="float32")  # 마지막 행 end_x, end_y

#     episodes.append(seq)
#     targets.append(target)
# print("에피소드 수 : ", len(episodes))

26902


100%|██████████| 26902/26902 [00:00<00:00, 60119.82it/s]


In [ ]:
test_meta = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

submission = submission.merge(test_meta, on="game_episode", how="left")

preds_x, preds_y = [], []

for _, row in tqdm(submission.iterrows(), total=len(submission)):
    g = pd.read_csv('../data' + row["path"][1:]).reset_index(drop=True)

    # 마지막 행 정보
    last_row = g.iloc[-1]
    last_phase = last_row["phase"]
    last_team = last_row["team_id"]

    # 마지막 phase 데이터
    phase_df = g[g["phase"] == last_phase]
    input_df = phase_df
    
    # -----------
    # case 1 : 마지막 phase 길이 >= 2
    # if len(phase_df) >= 2:
    #     input_df = phase_df
    # -----------
    # case 2 : 마지막 phase 길이 == 1
    # else:
    #     input_df = g
        # prev_df = g[g["phase"] < last_phase]
        # prev_df = prev_df[prev_df["team_id"] == last_team]

        # 만약 추론할 phase와 동일한 팀의 phase가 없을 때는 그냥 episode 전체 데이터를 활용
        # if len(prev_df) == 0:
        #     input_df = g
        # else:
        #     prev_phase = prev_df["phase"].max()
        #     input_df = prev_df[prev_df["phase"] == prev_phase]

    # 정규화된 좌표 준비
    sx = input_df["start_x"].values / 105.0
    sy = input_df["start_y"].values / 68.0
    ex = input_df["end_x"].values / 105.0
    ey = input_df["end_y"].values / 68.0
    
    coords = []
    for i in range(len(input_df)):
        # start는 항상 존재하므로 그대로 사용
        coords.append([sx[i], sy[i]])
        # 마지막 행은 end_x가 NaN이므로 자동으로 제외됨
        if i < len(input_df) - 1:
            coords.append([ex[i], ey[i]])

    seq = np.array(coords, dtype="float32")  # [T, 2]

    # x = torch.tensor(seq).unsqueeze(0).to(DEVICE)      # [1, T, 2]
    # length = torch.tensor([seq.shape[0]]).to(DEVICE)   # [1]

    # with torch.no_grad():
    #     pred = model(x, length).cpu().numpy()[0]       # [2], 정규화 좌표

    # preds_x.append(pred[0] * 105.0)
    # preds_y.append(pred[1] * 68.0)
print("Inference Done.")